In [4]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
from seq2cells.utils.anndata_utils import pseudo_bulk

In [2]:
data_path = "/gpfs/radev/project/ying_rex/tl688/seq2cells_data/multiome/10xmuleiome_seq_embeddings.h5ad"

In [5]:
anndata = ad.read(data_path)

In [11]:
anndata.layers['x_raw'] = anndata.X.copy()

In [14]:
anndata.obs

,feature_types,genome,chrom,chromStart,chromEnd,n_counts,highly_variable,batch,enf_set
HES4,Gene Expression,GRCh38,chr1,998961,998961,776.0,True,gene,train
ISG15,Gene Expression,GRCh38,chr1,1001137,1001137,5597.0,True,gene,train
TNFRSF18,Gene Expression,GRCh38,chr1,1203507,1203507,346.0,True,gene,train
TNFRSF4,Gene Expression,GRCh38,chr1,1211339,1211339,380.0,True,gene,valid
AJAP1,Gene Expression,GRCh38,chr1,4654608,4654608,47.0,True,gene,test
...,...,...,...,...,...,...,...,...,...
chrX:155767383-155767733,Peaks,GRCh38,chrX,155767383,155767733,126.0,False,ataac,valid
chrX:155820122-155820523,Peaks,GRCh38,chrX,155820122,155820523,174.0,False,ataac,train
chrX:155841301-155841724,Peaks,GRCh38,chrX,155841301,155841724,216.0,False,ataac,train
chrX:155880572-155882091,Peaks,GRCh38,chrX,155880572,155882091,3445.0,False,ataac,valid


In [22]:
anndata.var_names.unique()

Index(['AAACAGCCAATCCCTT-1', 'AAACAGCCAATGCGCT-1', 'AAACAGCCACCAACCG-1',
       'AAACAGCCAGGATAAC-1', 'AAACAGCCAGTTTACG-1', 'AAACAGCCATCCAGGT-1',
       'AAACATGCAAGGTCCT-1', 'AAACATGCACCGGCTA-1', 'AAACATGCACTTGTTC-1',
       'AAACATGCAGCAAGTG-1',
       ...
       'TTTGTGTTCGATAACC-1', 'TTTGTGTTCGGGACCT-1', 'TTTGTGTTCGTCAAGT-1',
       'TTTGTGTTCTCCATAT-1', 'TTTGTTGGTCAGGAAG-1', 'TTTGTTGGTGACATGC-1',
       'TTTGTTGGTGTTAAAC-1', 'TTTGTTGGTTAGGATT-1', 'TTTGTTGGTTGGTTAG-1',
       'TTTGTTGGTTTGCAGA-1'],
      dtype='object', name='cells', length=9631)

In [17]:
np.unique(anndata.var)

array([], dtype=float64)

In [26]:
anndata.obs["batch"].values

['gene', 'gene', 'gene', 'gene', 'gene', ..., 'ataac', 'ataac', 'ataac', 'ataac', 'ataac']
Length: 109145
Categories (2, object): ['gene', 'ataac']

In [27]:
anndata[anndata.obs["batch"] == "gene"]

View of AnnData object with n_obs × n_vars = 1999 × 9631
    obs: 'feature_types', 'genome', 'chrom', 'chromStart', 'chromEnd', 'n_counts', 'highly_variable', 'batch', 'enf_set'
    obsm: 'seq_embedding', 'seq_embeddings'
    layers: 'x_raw'

In [29]:
import torch
torch.randn(5, 6)

tensor([[ 0.8846, -1.8539, -1.8571, -0.1700,  0.2262,  2.0830],
        [ 0.1571,  0.8517, -0.6254,  0.5414, -1.0195, -0.2310],
        [ 0.5521,  0.1505,  0.7689, -0.5463, -0.6307, -0.2077],
        [ 1.5297, -0.4966,  0.4956,  0.2710,  0.9249, -0.3373],
        [-0.3356,  0.6925,  0.1537,  0.0027,  0.4008, -0.1217]])

In [ ]:
# sampled_adata_test = sampled_adata_test.T
sampled_adata_test.layers['x_raw'] = sampled_adata_test.X.copy()
sampled_adata_test = pseudo_bulk(sampled_adata_test,
            genes=list(sampled_adata_test.obs['ens_id']),
            cell_type_col='cell types',
            gene_col='ens_id',
            mode='mean',
            expr_threshold=0,
            mem_efficient_mode=True,
            layer='x_raw')